In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [42]:
agent_data = pd.read_csv('Data/datastorm_agent_data.csv')
policy_data = pd.read_csv("Data/datastorm_policy_data_short.csv")

In [43]:
policy_data

,main_holder_occupation_cd,policy_term,policy_payment_mode,policy_status,commencement_dt,next_due_dt,termination_dt,termination_reason,main_holder_gender,main_holder_dob,main_holder_entry_age,main_holder_smoker_flag,spouse_gender,spouse_dob,spouse_entry_age,spouse_smoker_flag,total_sum_assuared,premium_value,rider1_prem,rider1_sum_assuared,rider2_prem,rider2_sum_assuared,rider3_prem,rider3_sum_assuared,rider4_prem,rider4_sum_assuared,rider5_prem,rider5_sum_assuared,rider6_prem,rider6_sum_assuared,rider7_prem,rider7_sum_assuared,rider8_prem,rider8_sum_assuared,rider9_prem,rider9_sum_assuared,rider10_prem,rider10_sum_assuared,child1_dob,child1_gender,child2_dob,child2_gender,child3_dob,child3_gender,child4_dob,child4_gender,child5_dob,child5_gender,payment_method,main_holder_occupation,policy_snapshot_as_on,run_date,client_code,agent_code,product_code,product_name,policy_code
0,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20190131,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
1,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20200301,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
2,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20181231,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
3,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20191001,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
4,226.0,12,M,INFORCE,12/28/2018,3/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20190228,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9831,200.0,30,Q,INFORCE,7/7/2005,1/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20181231,20200701,C100533,AG106849,RE001,RETIREMENT,POL25553
9832,200.0,30,Q,INFORCE,7/7/2005,4/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200201,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9833,200.0,30,Q,INFORCE,7/7/2005,10/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20191001,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9834,200.0,30,Q,INFORCE,7/7/2005,1/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200101,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553


In [44]:
# Useful Columns
columns = ["main_holder_occupation_cd", "policy_term", "policy_payment_mode", ""]

In [61]:
snapshot_dates = np.sort(pd.unique(policy_data["policy_snapshot_as_on"]))

def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]

k = 6

snapshot_dates = list(divide_chunks(snapshot_dates, k))

print(snapshot_dates)

[array([20181231, 20190131, 20190228, 20190331, 20190430, 20190531],
      dtype=int64), array([20190630, 20190731, 20190831, 20190901, 20191001, 20191101],
      dtype=int64), array([20191201, 20200101, 20200201, 20200301, 20200401, 20200501],
      dtype=int64), array([20200601, 20200701], dtype=int64)]


In [73]:
# final_df = 

for clinet_name,client_data in policy_data.groupby(["client_code"]):
    client_data.sort_values(by=["policy_snapshot_as_on"], inplace=True)


    # for k in len(snapshot_dates):
    #     for i in snapshot_dates[1]:
    print(clinet_name)
    print(client_data.head(10))
    

         F   6/1/1977   
9797                     40                       N             F   6/1/1977   
9811                     40                       N             F   6/1/1977   
9799                     40                       N             F   6/1/1977   
9807                     40                       N             F   6/1/1977   

      spouse_entry_age spouse_smoker_flag  total_sum_assuared  premium_value  \
9806              30.0                  N               50000           5880   
9805              30.0                  N               50000           5880   
9815              30.0                  N               50000           5880   
9810              30.0                  N               50000           5880   
9801              30.0                  N               50000           5880   
9809              30.0                  N               50000           5880   
9797              30.0                  N               50000           5880   
9811          

In [52]:
# bins = [0, 1, 5, 10, 25, 50, 100]
# df['binned'] = pd.cut(df['percentage'], bins)